In [64]:
import pandas as pd
import numpy as np

## 1. Importing input and output variables

In [86]:
# Market variables 
input_market = pd.read_csv('input_market.csv')
input_market.rename(columns={input_market.columns[0]: "date" }, inplace = True)
input_market['date'] =  pd.to_datetime(input_market['date'])

# Stock variables
input_ret = pd.read_csv('input_ret.csv')
input_ret.rename(columns={input_ret.columns[0]: "date" }, inplace = True)
input_ret['date'] =  pd.to_datetime(input_ret['date'])

input_ret_cum = pd.read_csv('input_ret_cum.csv')
input_ret_cum.rename(columns={input_ret_cum.columns[0]: "date" }, inplace = True)
input_ret_cum['date'] =  pd.to_datetime(input_ret_cum['date'])

input_std = pd.read_csv('input_std.csv')
input_std.rename(columns={input_std.columns[0]: "date" }, inplace = True)
input_std['date'] =  pd.to_datetime(input_std['date'])

input_alpha = pd.read_csv('input_alpha.csv')
input_alpha.rename(columns={input_alpha.columns[0]: "date" }, inplace = True)
input_alpha['date'] =  pd.to_datetime(input_alpha['date'])

input_beta = pd.read_csv('input_beta.csv')
input_beta.rename(columns={input_beta.columns[0]: "date" }, inplace = True)
input_beta['date'] =  pd.to_datetime(input_beta['date'])

In [94]:
input_market

,date,Mkt,Mkt_cumret,Mkt_std
0,1963-01-01,0.0518,-0.019519,0.060939
1,1963-02-01,-0.0215,-0.059504,0.060879
2,1963-03-01,0.0331,-0.023687,0.061776
3,1963-04-01,0.0476,0.092369,0.059705
4,1963-05-01,0.0200,0.216527,0.052011
...,...,...,...,...
691,2020-08-01,0.0764,0.241916,0.069862
692,2020-09-01,-0.0362,0.177993,0.071767
693,2020-10-01,-0.0209,0.128434,0.072512
694,2020-11-01,0.1248,0.220562,0.079251


In [91]:
universe = pd.read_csv('universe.csv')
universe['date'] =  pd.to_datetime(universe['date'])
universe['permno'] = universe['permno'].astype(str)

In [92]:
universe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336000 entries, 0 to 335999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    336000 non-null  datetime64[ns]
 1   permno  336000 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 5.1+ MB


In [74]:
input_ret

,date,10006,10078,10095,10102,10104,10107,10108,10119,10137,...,93152,93174,93179,93223,93246,93295,93312,93422,93429,93436
0,1963-01-01,0.047002,NaN,NaN,0.041475,NaN,NaN,NaN,NaN,0.053846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1963-02-01,0.038700,NaN,NaN,-0.004425,NaN,NaN,NaN,NaN,-0.004866,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1963-03-01,-0.009009,NaN,NaN,0.047191,NaN,NaN,NaN,NaN,-0.012714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1963-04-01,0.084848,NaN,NaN,-0.021459,NaN,NaN,NaN,NaN,0.017500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1963-05-01,0.091620,NaN,NaN,-0.008772,NaN,NaN,NaN,NaN,-0.027518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2020-08-01,NaN,NaN,NaN,NaN,0.031921,0.102580,NaN,NaN,NaN,...,NaN,0.146067,0.042345,NaN,0.205610,NaN,0.110609,-0.115646,NaN,0.741452
692,2020-09-01,NaN,NaN,NaN,NaN,0.043341,-0.067397,NaN,NaN,NaN,...,NaN,-0.169550,-0.064687,NaN,0.019265,NaN,-0.050220,-0.305538,NaN,-0.139087
693,2020-10-01,NaN,NaN,NaN,NaN,-0.056114,-0.037370,NaN,NaN,NaN,...,NaN,0.011111,0.118610,NaN,0.085261,NaN,-0.021480,-0.003102,NaN,-0.095499
694,2020-11-01,NaN,NaN,NaN,NaN,0.028694,0.060058,NaN,NaN,NaN,...,NaN,0.239698,0.128435,NaN,0.025934,NaN,0.165653,0.788889,NaN,0.462736


In [95]:
# Assuming that in keras LSTM the time flows from top to bottom
# Assuming the lookback for features is 12

res2 = []
for (dt, permno), _data in universe.groupby(['date', 'permno']):
        # Construct LSTM features
        market = input_market.loc[
            input_market['date'] <= dt].tail(12)

        stock_ret = input_ret.loc[
            input_ret["date"] <= dt
            ][["date", permno]].tail(12)

        merged = market.merge(stock_ret, on="date")
        features = merged[["Mkt", "Mkt_cumret", "Mkt_std", permno]].values

        res2.append(features)

res2 = np.array(res2)

In [99]:
res2

array([[[ 1.80000000e-02,  2.26851390e-01,  2.27291096e-02,
          4.37740000e-02],
        [ 1.72000000e-02,  2.07969446e-01,  2.21887547e-02,
          0.00000000e+00],
        [ 3.90000000e-03,  1.57579731e-01,  2.00283209e-02,
          3.63600000e-03],
        ...,
        [ 2.90000000e-03,  1.81207776e-01,  1.13370939e-02,
         -2.32760000e-02],
        [ 3.40000000e-03,  1.60618765e-01,  1.14777651e-02,
         -5.63800000e-02],
        [ 3.82000000e-02,  1.75106691e-01,  1.32400105e-02,
          5.34590000e-02]],

       [[ 1.80000000e-02,  2.26851390e-01,  2.27291096e-02,
         -2.82610000e-02],
        [ 1.72000000e-02,  2.07969446e-01,  2.21887547e-02,
          7.01360000e-02],
        [ 3.90000000e-03,  1.57579731e-01,  2.00283209e-02,
         -4.65120000e-02],
        ...,
        [ 2.90000000e-03,  1.81207776e-01,  1.13370939e-02,
         -4.38600000e-03],
        [ 3.40000000e-03,  1.60618765e-01,  1.14777651e-02,
         -4.00890000e-02],
        [ 3.820

In [85]:
input_ret

,date,10006,10078,10095,10102,10104,10107,10108,10119,10137,...,93152,93174,93179,93223,93246,93295,93312,93422,93429,93436
0,1963-01,0.047002,NaN,NaN,0.041475,NaN,NaN,NaN,NaN,0.053846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1963-02,0.038700,NaN,NaN,-0.004425,NaN,NaN,NaN,NaN,-0.004866,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1963-03,-0.009009,NaN,NaN,0.047191,NaN,NaN,NaN,NaN,-0.012714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1963-04,0.084848,NaN,NaN,-0.021459,NaN,NaN,NaN,NaN,0.017500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1963-05,0.091620,NaN,NaN,-0.008772,NaN,NaN,NaN,NaN,-0.027518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2020-08,NaN,NaN,NaN,NaN,0.031921,0.102580,NaN,NaN,NaN,...,NaN,0.146067,0.042345,NaN,0.205610,NaN,0.110609,-0.115646,NaN,0.741452
692,2020-09,NaN,NaN,NaN,NaN,0.043341,-0.067397,NaN,NaN,NaN,...,NaN,-0.169550,-0.064687,NaN,0.019265,NaN,-0.050220,-0.305538,NaN,-0.139087
693,2020-10,NaN,NaN,NaN,NaN,-0.056114,-0.037370,NaN,NaN,NaN,...,NaN,0.011111,0.118610,NaN,0.085261,NaN,-0.021480,-0.003102,NaN,-0.095499
694,2020-11,NaN,NaN,NaN,NaN,0.028694,0.060058,NaN,NaN,NaN,...,NaN,0.239698,0.128435,NaN,0.025934,NaN,0.165653,0.788889,NaN,0.462736


In [84]:
input_market

,date,Mkt,Mkt_cumret,Mkt_std
0,1963-01-01,0.0518,-0.019519,0.060939
1,1963-02-01,-0.0215,-0.059504,0.060879
2,1963-03-01,0.0331,-0.023687,0.061776
3,1963-04-01,0.0476,0.092369,0.059705
4,1963-05-01,0.0200,0.216527,0.052011
...,...,...,...,...
691,2020-08-01,0.0764,0.241916,0.069862
692,2020-09-01,-0.0362,0.177993,0.071767
693,2020-10-01,-0.0209,0.128434,0.072512
694,2020-11-01,0.1248,0.220562,0.079251


In [101]:
# Assume that in keras LSTM the time flows from top to bottom
# Assume the lookback for features is 12

res2 = []
for (dt, permno), _data in universe.groupby(['date', 'permno']):
        # Construct LSTM features
        market = input_market.loc[
            input_market['date'] <= dt].tail(12)

        stock_alpha = input_alpha.loc[
            input_alpha['date'] <= dt
            ][["date", permno]].tail(12)

        stock_beta = input_beta.loc[
            input_beta["date"] <= dt
            ][["date", permno]].tail(12)

        stock_std = input_std.loc[
            input_std["date"] <= dt
            ][["date", permno]].tail(12)

        stock_ret = input_ret.loc[
            input_ret["date"] <= dt
            ][["date", permno]].tail(12)
        
        stock_ret_cum = input_ret_cum.loc[
            input_ret_cum["date"] <= dt
            ][["date", permno]].tail(12)

        data_frames = [input_market, input_alpha, input_beta, input_std, input_ret, input_ret_cum]
        merged = pd.concat(data_frames, join='outer', axis=1)

        features = merged[["Mkt", "Mkt_cumret", "Mkt_std", "stock_alpha", "stock_beta", "stock_std", "stock_ret", "stock_ret_cum", permno]].values

        #res[dt][permno] = features
        res2.append(features)
        #res2[dt][permno] = features

res2 = np.array(res2)

KeyError: "['stock_betastock_std', 'stock_ret', 'stock_ret_cum', 'stock_alpha'] not in index"